In [1]:
import pickle
import string

from babel.dates import format_date

import altair as alt
import numpy as np
import pandas as pd
import re
import spacy

from bs4 import BeautifulSoup
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from gensim.corpora.dictionary import Dictionary

from utils.data import notes, resultats

In [2]:
pd.options.display.max_colwidth = 200
DEFAULT_WIDTH = 800

_ = alt.data_transformers.disable_max_rows()

In [4]:
with open("canebiere.pickle", "rb") as src:
    df: pd.DataFrame = pickle.load(src)

blaah = df[df['Auteur'] == 'Blaah'].reset_index()
matchs = resultats(blaah)

# L'OM de Blaah

## Nombre de buts dans les matches

In [5]:
matchs["Nb Buts"] = matchs["Buts_R"] + matchs["Buts_V"]

In [30]:
alt.Chart(matchs[["Nb Buts"]]).mark_bar().encode(
    x=alt.X("Nb Buts:O", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("count():Q", title="Matchs"),
    color=alt.Color("Nb Buts:O", scale=alt.Scale(scheme="greens")),
    tooltip=["Nb Buts:O", alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

## Nombre de buts de l'OM

In [28]:
alt.Chart(matchs[["Buts_OM"]]).mark_bar().encode(
    x=alt.X("Buts_OM:O", title="Buts de l'OM", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("count():Q", title='Matchs'),
    color="Buts_OM:O",
    tooltip=[alt.Tooltip("Buts_OM:O", title="Buts de l'OM"), alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

## Nombre de buts de l'adversaire

In [29]:
alt.Chart(matchs[["Buts_Adversaire"]]).mark_bar().encode(
    x=alt.X("Buts_Adversaire:O", title="Buts de l'adversaire", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("count():Q", title="Matchs"),
    color=alt.Color("Buts_Adversaire:O", scale=alt.Scale(scheme="reds")),
    tooltip=[alt.Tooltip("Buts_Adversaire:O", title="Buts de l'OM"), alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

## Différence de But par Adversaire

In [9]:
by_adv = matchs.groupby("Adversaire").size()

In [10]:
# Il va falloir traiter les doublons...

# * ASSE / Saint-Etienne / Saint-Étienne
# * TFC / Toulouse
# * MHSC / Montpellier
# * Gazélec / Gazéléc
# * Konyaspot / Konyasport
# * Salzbourg / Salzburg
# * Athletic / Bilbao

In [21]:
normalize = [
    {"dups": ["ASSE", "Saint-Etienne", "Saint-Étienne"], "norm": "Saint-Étienne"},
    {"dups": ["TFC", "Toulouse"], "norm": "Toulouse"},
    {"dups": ["MHSC", "Montpellier"], "norm": "Montpellier"},
    {"dups": ["Gazélec", "Gazéléc"], "norm": "Gazélec Ajaccio"},
    {"dups": ["Konyaspor", "Konyasport"], "norm": "Konyaspor"},
    {"dups": ["Salzbourg", "Salzburg"], "norm": "Salzburg"},
    {"dups": ["Athletic", "Bilbao"], "norm": "Athletic Bilbao"},
    {"dups": ["Atlético"], "norm": "Atletico Madrid"},
    {"dups": ["PAOK", 'PAOK Salonique'], "norm": "PAOK"}
]

In [22]:
matchs["Publications"] = 1

In [23]:
for dedup in normalize:
    matchs.loc[matchs["Adversaire"].isin(dedup["dups"]), "Adversaire"] = dedup["norm"]

In [24]:
by_adv = matchs.groupby("Adversaire").agg({"Publications": "sum", "Buts_OM": "sum", "Buts_Adversaire": "sum"})
by_adv.reset_index(inplace=True)

In [25]:
by_adv["Différence de Buts"] = by_adv["Buts_OM"] - by_adv["Buts_Adversaire"]

In [26]:
slider = alt.binding_range(min=by_adv["Publications"].min(), max=by_adv["Publications"].max(), step=1, name="Nombre Minimum de Matchs: ")
select_nb_matches = alt.selection_single(name="matches", fields=["Publications"], bind=slider, init={"Publications": 1})

alt.Chart(by_adv).mark_bar().encode(
    y="Différence de Buts",
    x=alt.X(
        "Adversaire:O", 
        sort=alt.EncodingSortField(field="Différence de Buts", order="ascending"),
        axis=alt.Axis(labelAngle=-45)
    ),
    color=alt.Color("Différence de Buts", scale=alt.Scale(scheme="redyellowgreen")),
    tooltip=["Adversaire", "Différence de Buts", alt.Tooltip("Publications", title="Nb Matches")]
).add_selection(
    select_nb_matches
).transform_filter(
    "datum.Publications >= matches_Publications"
). properties(height=600,width=1200)

alt.Chart(...)

## Différentiel Victoire / Défaite par Adversaire

In [18]:
matchs["Victoire"] = matchs["Buts_OM"] > matchs["Buts_Adversaire"]
matchs["Défaite"] =  matchs["Buts_OM"] < matchs["Buts_Adversaire"]
matchs["Nul"] =  matchs["Buts_OM"] == matchs["Buts_Adversaire"]

In [19]:
by_adv = matchs.groupby("Adversaire").agg({"Publications": "sum", "Victoire": "sum", "Défaite": "sum", "Nul": "sum"})
by_adv.reset_index(inplace=True)
by_adv["Différence Victoire / Défaite"] = by_adv["Victoire"].astype(int) - by_adv["Défaite"].astype(int)

In [20]:
chart = alt.Chart(by_adv).mark_bar().encode(
    y="Différence Victoire / Défaite",
    x=alt.X(
        "Adversaire:O", 
        sort=alt.EncodingSortField(field="Différence Victoire / Défaite"),
        axis=alt.Axis(labelAngle=-45)
    ),
    color=alt.Color("Différence Victoire / Défaite", scale=alt.Scale(domain=[-10, 10], scheme="redyellowgreen")),
    tooltip=["Adversaire", "Victoire", "Nul", "Défaite", alt.Tooltip("Publications", title="Nb Matches")]
).add_selection(
    select_nb_matches
).transform_filter(
    "datum.Publications >= matches_Publications"
). properties(height=600,width=1200)

chart

alt.Chart(...)